In [ ]:
import sys
import os
sys.path.extend(['Fyler', 'EvalFyler', 'Lib', 'Codes'])

In [ ]:
# Set environment variables
keep_defaults = False
def set_vars(key, value):
    if keep_defaults:
        return os.environ.setdefault(key, value)
    else:
        os.environ[key] = value
        return os.environ[key]

############################################################
# EDIT THIS STRING to point to your local ACHD mount path. #
############################################################
print(set_vars('ACHD', '/home/tmill/mnt/ACHD'))

print(set_vars('DATA_ROOT', '/home/tmill/mnt/ACHD/tim_ACHD/data'))

print(set_vars('DOWNSTREAM', '/home/tmill/mnt/ACHD/angus_ACHD/datasets/pretrained_downstream/datasets_corrected_stratified/'))
#print(os.environ['DOWNSTREAM'])

In [ ]:
from EvalFyler import fyler_fextract

In [ ]:
# Get downstream config
example_downstream_cfg_path = 'example_cfgs/experiment-cc-TGA_cnlpt-rest.cfg'
with open(example_downstream_cfg_path, 'r', encoding='utf8') as dcfg_fd:
    print(dcfg_fd.read())

In [ ]:
# Downstream classifier training
_ = fyler_fextract.train_model(
    gpu=-1, 
    model_class="cnlpt_rest",
    model_dir="models",
    out_dir=None, 
    cfg_path=example_downstream_cfg_path,
)

In [ ]:
import configparser
import json


cfg_fn = 'example_cfgs/experiment-cc-TGA_cnlpt-rest.cfg'
cfg = configparser.ConfigParser()
cfg.read(cfg_fn)
print(cfg.get('model', 'param_grid'))
json.loads(cfg.get('model', 'param_grid'))


#### What this looks like at the batch level:

```sh
$ # generate the different configurations for the encoders and the downstream tasks using base config files
$ # gen_experiments.py currently has the phenotypes, window sizes, and minimum count parameter grid hard-coded
$ python Fyler/gen_experiments.py pretrain \
    path/to/base_config.cfg \
    path/to/cfgs
$ python Fyler/gen_experiments.py fextract \
    path/to/downstream/base_config.cfg \
    path/to/downstream/cfgs
$ # generate the data into the directory specified in the config file
$ python Fyler/experiments.py data \
    path/to/cfgs \
    --exclude path/to/excluded-cfgs.txt \
    -p num_cores \
    -o path/to/data/log/dir
$ # train Fyler encoders
$ python Fyler/fyler_bow.py batch \
    path/to/cfgs \
    path/to/data/dirs \
    path/to/fyler/encoders \
    --exclude path/to/excluded-downstream-cfgs.txt \
    -g num_gpus \
    -o path/to/fyler/log/dir
$ # train downstream classifiers
$ python EvalFyler/fyler_fextract.py batch \
    path/to/downstream/cfgs \
    path/to/fyler/encoders \
    --model_class fyler \
    --exclude path/to/excluded-downstream-cfgs.txt \
    -g num_gpus \
    -o path/to/downstream/log/dir
```